#### USCRN Data 

In [216]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import yaml 

with open ("sources.yaml", "r") as yaml_file:
  sources = yaml.load(yaml_file)

/tmp/ipykernel_41028/798438717.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  sources = yaml.load(yaml_file)


In [243]:
header_url = sources['USCRN']['headers']
header_response = requests.get(header_url)
header_soup = BeautifulSoup(header_response.content, "html.parser")

columns = str(header_soup).split("\n")[1].strip(" ").split(" ")
columns = list(map(lambda x: str.lower(x), columns))
columns.insert(0,'station_location')

In [218]:
base_url = sources["USCRN"]["index"]
base_response = requests.get(base_url)
base_soup = BeautifulSoup(base_response.content, "html.parser")

In [219]:
import re

links = base_soup.find_all("a") # 'links' in this notebook will refer to <a> elements, not urls
years = [str(x).zfill(1) for x in range(2000,2024)]
year_links = [link for link in links if link['href'].rstrip('/') in years]

file_urls = []
for year_link in year_links: 
  year_url = base_url + year_link.get("href")
  response = requests.get(year_url) 
  soup = BeautifulSoup(response.content, 'html.parser')
  file_links = soup.find_all('a', href=re.compile(r'AK.*\.txt'))
  if file_links:
    new_file_urls = [year_url + link.getText() for link in file_links]
    file_urls.extend(new_file_urls)

In [249]:
rows = []
for url in file_urls:
  response = requests.get(url)
  station_location = re.search("_[A-Z]{1}[a-z]+_", url)[0].strip("_")
  soup = BeautifulSoup(response.content,'html.parser')
  soup_lines = [station_location + " " + line for line in str(soup).split("\n")]
  new_rows = [re.split('\s+', row) for row in soup_lines]
  rows.extend(new_rows)

In [252]:
df = pd.DataFrame(rows, columns=columns)

In [247]:
# # Demo of loop 
# file_response = requests.get(file_urls[0])
# station_location = re.search("_[A-Z]{1}[a-z]+_", file_urls[0])[0].strip("_")
# soup = BeautifulSoup(file_response.content, 'html.parser')
# soup_lines = [station_location + " " + line for line in str(soup).split("\n")]
# rows = [re.split('\s+', row) for row in soup_lines]
# # df = pd.DataFrame(rows,columns=columns)
# # df.insert(0, 'station_location', station_location)

In [248]:
rows

[['Fairbanks',
  '26494',
  '20020809',
  '2200',
  '20020809',
  '1300',
  '1.001',
  '-147.51',
  '64.97',
  '-9999.0',
  '-9999.0',
  '-9999.0',
  '-9999.0',
  '-9999.0',
  '489',
  '3',
  '-99999',
  '0',
  '-99999',
  '0',
  'R',
  '19.4',
  '3',
  '-9999.0',
  '0',
  '-9999.0',
  '0',
  '0',
  '3',
  '-99.000',
  '-99.000',
  '-99.000',
  '-99.000',
  '-99.000',
  '-9999.0',
  '-9999.0',
  '-9999.0',
  '-9999.0',
  '-9999.0'],
 ['Fairbanks',
  '26494',
  '20020809',
  '2300',
  '20020809',
  '1400',
  '1.001',
  '-147.51',
  '64.97',
  '10.0',
  '11.1',
  '12.0',
  '10.0',
  '-9999.0',
  '99',
  '0',
  '-99999',
  '0',
  '-99999',
  '0',
  'R',
  '14.6',
  '0',
  '-9999.0',
  '0',
  '-9999.0',
  '0',
  '0',
  '3',
  '-99.000',
  '-99.000',
  '-99.000',
  '-99.000',
  '-99.000',
  '-9999.0',
  '-9999.0',
  '-9999.0',
  '-9999.0',
  '-9999.0'],
 ['Fairbanks',
  '26494',
  '20020810',
  '0000',
  '20020809',
  '1500',
  '1.001',
  '-147.51',
  '64.97',
  '12.0',
  '10.5',
  '12.0',


In [235]:
df.tail(5).to_csv("sample.csv", index=False)
df.to_csv("sample.csv", mode='a', index=False, header=False)

In [222]:
df.iloc[3459,:]

station_location     Fairbanks
wbanno                        
utc_date                  None
utc_time                  None
lst_date                  None
lst_time                  None
crx_vn                    None
longitude                 None
latitude                  None
t_calc                    None
t_hr_avg                  None
t_max                     None
t_min                     None
p_calc                    None
solarad                   None
solarad_flag              None
solarad_max               None
solarad_max_flag          None
solarad_min               None
solarad_min_flag          None
sur_temp_type             None
sur_temp                  None
sur_temp_flag             None
sur_temp_max              None
sur_temp_max_flag         None
sur_temp_min              None
sur_temp_min_flag         None
rh_hr_avg                 None
rh_hr_avg_flag            None
soil_moisture_5           None
soil_moisture_10          None
soil_moisture_20          None
soil_moi

In [92]:
import re
x = year_soups[1].find_all('a', href=re.compile(r'.*NC.*\.txt'))
x

[<a href="CRNH0203-2001-NC_Asheville_8_SSW.txt">CRNH0203-2001-NC_Asheville_8_SSW.txt</a>,
 <a href="CRNH0203-2001-NC_Asheville_13_S.txt">CRNH0203-2001-NC_Asheville_13_S.txt</a>]

In [99]:
{a.strip("_") for a in re.findall(r'_[A-Z]{1}[a-z]+_',x[0].getText())}

{'Asheville'}